In [1]:
import ssl
import certifi
import osmnx as ox
import networkx as nx
import folium
from geopy.geocoders import Nominatim

# -------------------------
# Setup: SSL and Geolocator
# -------------------------
ssl_context = ssl.create_default_context(cafile=certifi.where())
geolocator = Nominatim(user_agent='wichita_nav', ssl_context=ssl_context)

# ----------------------------------------
# Download and preprocess road network data
# ----------------------------------------
G_multi = ox.graph_from_place('Wichita, Kansas, USA', network_type='drive')
ox.add_edge_speeds(G_multi)
ox.add_edge_travel_times(G_multi)

# Convert MultiGraph to DiGraph, keeping only the fastest edge between nodes
def multigraph_to_digraph(G_multi):
    G_simple = nx.DiGraph()
    # Copy nodes with attributes
    G_simple.add_nodes_from(G_multi.nodes(data=True))
    # For each edge, keep the one with the least travel_time
    for u, v, data in G_multi.edges(data=True):
        if G_simple.has_edge(u, v):
            if data.get('travel_time', float('inf')) < G_simple[u][v].get('travel_time', float('inf')):
                G_simple[u][v].update(data)
        else:
            G_simple.add_edge(u, v, **data)
    return G_simple

G = multigraph_to_digraph(G_multi)
G.graph["crs"] = G_multi.graph.get("crs")

# -------------------------------
# Helper functions for visualization
# -------------------------------
def plot_route(G, route, route_map, color='blue'):
    route_points = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]
    folium.PolyLine(route_points, color=color, weight=4.5, opacity=0.7).add_to(route_map)

def get_nearest_node(address): 
    location = geolocator.geocode(address, timeout=5)
    if location:
        print(f"Geocoded Address: {address} -> ({location.latitude}, {location.longitude})")
        return ox.distance.nearest_nodes(G, X=location.longitude, Y=location.latitude)
    else: 
        print(f"Failed to geocode address: {address}")
        return None

def get_travel_time(G, route): 
    travel_time = 0
    for u, v in zip(route[:-1], route[1:]):
        travel_time += G[u][v]['travel_time']
    return travel_time

# Highway score: proportion of edges that are main roads
def highway_score(G, route): 
    main_road_types = ['motorway', 'trunk', 'primary', 'secondary']
    edges = list(zip(route, route[1:]))
    count = 0
    for u, v in edges: 
        edge_data = G.get_edge_data(u, v)
        if edge_data: 
            highway = edge_data.get('highway', None)
            if isinstance(highway, list):
                if any(h in main_road_types for h in highway):
                    count += 1
            else:
                if highway in main_road_types:
                    count += 1
    return count / len(edges) if edges else 0

# -------------------------------
# Define custom weight functions
# -------------------------------
# 1. Fastest route: use existing travel_time weight.
# 2. Highway Priority: add a penalty for non-highway segments.
def highway_priority_weight(u, v, data):
    main_road_types = ['motorway', 'trunk', 'primary', 'secondary']
    highway = data.get('highway', None)
    if isinstance(highway, list):
        is_highway = any(h in main_road_types for h in highway)
    else:
        is_highway = highway in main_road_types
    penalty = 0 if is_highway else 60  # add a 60-second penalty for non-highway segments
    return data['travel_time'] + penalty

# 3. Balanced Route: give a bonus for highway segments (subtract time) but avoid negative weights.
def balanced_route_weight(u, v, data):
    main_road_types = ['motorway', 'trunk', 'primary', 'secondary']
    highway = data.get('highway', None)
    if isinstance(highway, list):
        is_highway = any(h in main_road_types for h in highway)
    else:
        is_highway = highway in main_road_types
    bonus = 30 if is_highway else 0  # subtract 30 seconds if on a highway
    return max(data['travel_time'] - bonus, 1)  # ensure cost is at least 1 second

# -------------------------------
# Define start and end points
# -------------------------------
start_address = "7331 Ayesbury Cir, Wichita KS"
# end_address = "1220 N Tyler Rd, Wichita, KS"
end_address = "123 Douglas Ave, Wichita, KS"

start_node = get_nearest_node(start_address)
end_node = get_nearest_node(end_address)

# -------------------------------
# Compute routes using the three algorithms
# -------------------------------
# Fastest Route (using travel_time)
fastest_route = nx.shortest_path(G, start_node, end_node, weight='travel_time')

# Highway Priority Route (penalize non-highway segments)
highway_route = nx.shortest_path(G, start_node, end_node, weight=highway_priority_weight)

# Balanced Route (bonus for highways)
balanced_route = nx.shortest_path(G, start_node, end_node, weight=balanced_route_weight)

# -------------------------------
# Collect routes and compute metrics
# -------------------------------
routes = [
    {"name": "Fastest", "route": fastest_route, "color": "blue"},
    {"name": "Highway Priority", "route": highway_route, "color": "red"},
    {"name": "Balanced", "route": balanced_route, "color": "green"}
]

for r in routes:
    t = get_travel_time(G, r["route"])
    h = highway_score(G, r["route"])
    print(f"{r['name']} Route: Travel Time = {t:.1f} sec, Highway Score = {h:.2f}")

# -------------------------------
# Visualize routes on Folium map
# -------------------------------
location_wichita = geolocator.geocode("Wichita, Kansas, USA")
route_map = folium.Map(location=[location_wichita.latitude, location_wichita.longitude], zoom_start=12)

for r in routes:
    plot_route(G, r["route"], route_map, color=r["color"])

# Display the interactive map (in Jupyter, the map will render in the output cell)
route_map


Geocoded Address: 7331 Ayesbury Cir, Wichita KS -> (37.727137306122444, -97.25127428571429)
Geocoded Address: 123 Douglas Ave, Wichita, KS -> (37.68591687755102, -97.33855526530613)
Fastest Route: Travel Time = 693.0 sec, Highway Score = 0.71
Highway Priority Route: Travel Time = 736.4 sec, Highway Score = 0.91
Balanced Route: Travel Time = 808.8 sec, Highway Score = 0.90
